In [1]:
import cv2
import numpy as np
import math

In [2]:
path1 = r'imgs/IMG_20240510_172748.jpg'
path2 = r'imgs/IMG_20240510_172837.jpg'
path3 = r'imgs/IMG_20240510_172930.jpg'

In [29]:
def getCountorsBT(frame_orig, gray = False, draw = True):
    frame = frame_orig.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    

    cnt = [c[0] for c in cnts[:1]]
    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # Draw the center of the contour
        # cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
        # contours = [c[0] for c in cnt]

        cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)

        ellipse = cv2.fitEllipse(cnt[0])

        (cx,cy), (d1,d2), angle = ellipse
        print(int(cx), int(cy))
        # cv2.circle(frame_orig, (int(cx), int(cy)), 5, (255, 255, 255), -1)
        # cv2.imshow('center', frame_orig)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
                

        ellipse_img = frame_orig.copy()
        cv2.ellipse(ellipse_img, (int(cx),int(cy)), (int(d1/2),int(d2/2)), angle, 0, 360, (0,0,255), 4)

        if angle > 90:
            angle = angle - 90
        else:
            angle = angle + 90
        print('center: (', cx,cy, ')', 'diameters: (', d1, d2, ')', 'angle:', angle)

        r1 = min(d1,d2)/2
        r2 = max(d1,d2)/2

        x1 = np.intp( cx + math.cos(math.radians(angle))*r2 )
        y1 = np.intp( cy + math.sin(math.radians(angle))*r2 )
        x2 = np.intp( cx + math.cos(math.radians(angle+180))*r2 )
        y2 = np.intp( cy + math.sin(math.radians(angle+180))*r2 )
        x3 = np.intp( cx + math.cos(math.radians(angle+90))*r1 )
        y3 = np.intp( cy + math.sin(math.radians(angle+90))*r1 )
        x4 = np.intp( cx + math.cos(math.radians(angle+270))*r1 )
        y4 = np.intp( cy + math.sin(math.radians(angle+270))*r1 )

        input_pts = np.float32([[x1,y1], [x2,y2], [x3,y3], [x4,y4]])

        # output points from circle of radius = max semi-major radii
        ox1 = np.intp( cx + math.cos(math.radians(angle))*r2 )
        oy1 = np.intp( cy + math.sin(math.radians(angle))*r2 )
        ox2 = np.intp( cx + math.cos(math.radians(angle+180))*r2 )
        oy2 = np.intp( cy + math.sin(math.radians(angle+180))*r2 )
        ox3 = np.intp( cx + math.cos(math.radians(angle+90))*r2 )
        oy3 = np.intp( cy + math.sin(math.radians(angle+90))*r2 )
        ox4 = np.intp( cx + math.cos(math.radians(angle+270))*r2 )
        oy4 = np.intp( cy + math.sin(math.radians(angle+270))*r2 )

        cx = np.intp(cx)
        cy = np.intp(cy)
        r2 = np.intp(r2)
        print(int(cx), int(cy))
        print((ox1, oy1), (ox2, oy2), (ox3, oy3), (ox4, oy4))
        # cv2.circle(frame_orig, (int(cx), int(cy)), 5, (255, 0, 255), -1)
        # cv2.imshow('center', frame_orig)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
        # cv2.waitKey(1)
#         ellipse_pts = ellipse.copy()

# # draw white circle on copy of ellipse
#         cv2.circle(ellipse_pts, (cx,cy), r2, (255,255,255), 4)

        output_pts = np.float32([[ox1,oy1], [ox2,oy2], [ox3,oy3], [ox4,oy4]])

        h = cv2.getPerspectiveTransform(input_pts, output_pts)

        # warp image
        result = cv2.warpPerspective(frame, h, (frame.shape[1],frame.shape[0]))

        cv2.circle(result, (ox1, oy1), 5, (0, 255, 0), -1)
        cv2.circle(result, (ox2, oy2), 5, (0, 255, 0), -1)
        cv2.circle(result, (ox3, oy3), 5, (0, 255, 0), -1)
        cv2.circle(result, (ox4, oy4), 5, (0, 255, 0), -1)

        cv2.line(result, (ox1, oy1), (ox2, oy2), (0, 255, 0), 5)
        cv2.line(result, (ox3, oy3), (ox4, oy4), (0, 255, 0), 5)

        # cv2.ellipse(frame, ellipse,(0,255,0), 10)
        # cv2.circle(frame, (int(cx), int(cy)), 7, (0, 255, 255), -1)
        # for i in range(len(cnt)):
        #     print(i)
        #     x, y, w, h = cnt[i][4]
        #     frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)



    cnt = cnt[0]
    l = tuple(cnt[cnt[:,:,0].argmin()][0])
    r = tuple(cnt[cnt[:,:,0].argmax()][0])
    t = tuple(cnt[cnt[:,:,1].argmin()][0])
    b = tuple(cnt[cnt[:,:,1].argmax()][0])
    print(cv2.contourArea(cnt[0]) / float(frame.shape[0] * frame.shape[1]))

    return result, ellipse_img, cnt, (l, r, t, b), (d1/2, d2/2)

In [30]:
def getCountorsB(frame_orig, gray = False, draw = True):
    frame = frame_orig.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    

    cnt = [c[0] for c in cnts[:1]]
    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # Draw the center of the contour
        cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
        # contours = [c[0] for c in cnt]

        cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)
 
    cnt = cnt[0]
    l = tuple(cnt[cnt[:,:,0].argmin()][0])
    r = tuple(cnt[cnt[:,:,0].argmax()][0])
    t = tuple(cnt[cnt[:,:,1].argmin()][0])
    b = tuple(cnt[cnt[:,:,1].argmax()][0])
    print(cv2.contourArea(cnt[0]) / float(frame.shape[0] * frame.shape[1]))

    cv2.circle(frame, l, 5, (0, 255, 0), -1)
    cv2.circle(frame, r, 5, (0, 255, 0), -1)
    cv2.circle(frame, t, 5, (0, 255, 0), -1)
    cv2.circle(frame, b, 5, (0, 255, 0), -1)

    cv2.line(frame, l, r, (0, 255, 0), 5)
    cv2.line(frame, t, b, (0, 255, 0), 5)
    return frame, cnt, (l, r, t, b)

In [37]:
img = cv2.imread(path3)

w, h = img.shape[1]//2, img.shape[0]//2

img = cv2.resize(img, (w, h))

result, ellipse_img,  cnts1, extreme_pts, radiuses = getCountorsBT(img)
cv2.imshow('Orig', img)
cv2.imshow('Ellipse', ellipse_img)
cv2.imshow('Circle', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

# result,  cnts1, extreme_pts = getCountorsBT(img)
# cv2.imshow('orig', img)
# cv2.imshow('contours', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)
# cv2.waitKey(1)
# cv2.waitKey(1)
# cv2.waitKey(1)

1244 908
center: ( 1244.4505615234375 908.25146484375 ) diameters: ( 1359.5909423828125 1393.1175537109375 ) angle: 169.0656509399414
1244 908
(560, 1040) (1928, 776) (1112, 224) (1376, 1592)
0.0


-1

In [59]:
def getCountors(frame_orig):
    frame = frame_orig.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    

    cnt = [c[0] for c in cnts[:1]]
    bbox = [c[4] for c in cnts[:1]]
    print(bbox)
        # M = cv2.moments(cnt[0])
        # cX = int(M["m10"] / M["m00"])
        # cY = int(M["m01"] / M["m00"])
        # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # Draw the center of the contour
        # cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
    # cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)
    x, y, w, h = bbox[0]
    # cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 5)
    frame = frame[y:y+h, x:x+w]

    frame = cv2.resize(frame, (1000, 1000))
 
    # frame = cv2.circle(frame, (500, 500), 10, (0, 255, 0), -1)

    return frame

In [60]:
img = cv2.imread(path2)

w, h = img.shape[1]//3, img.shape[0]//3

img = cv2.resize(img, (w, h))

img = getCountors(img)
cv2.imshow('largest contour', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

[(447, 155, 796, 785)]


-1